# Exercise 3)
    We are going to decrypte the message from Alice.
    All we have is the Alice's and Bob's public keys noted g_A and g_B.
    And the group that they are using K* = (Z/pZ)* with p is prime number.
    We also know the generator of this group that they used, noted g

    To decrypte each bloc of message (u, v), we need to find the secret key of Bob coz Bob is the receiver. Let's note a this secret key. 
    We know that, in Elgamal cryptosystem, g^a = g_B over K*. So we need use the algorithm of Silver Pohlig Helmann to resolve this equation. 
    When we find this secret key, we just need to compute ((u^(-a)%p)*v)%p to find the decrypted message.
    To make message readable, we just decode the decrypted message that we found into letter.

    Below is the implementation of all algorithm that we need to find the readable message. We are going to use the factorint function from the sympy to factor a large number.
    

In [28]:
from sympy import factorint
import math

## Silver Pohlig Helmann

In [20]:
def silver_pohlig_hellman(g: int, h: int, p: int, n: int) -> int:
    """
        Solve g^x ≡ h mod p for a cyclic group K with order n.
        g: generator of K
        h: the target value
        p: a prime number for modulo calculus
        n: the groupe order
        Return discret Logarithm x. (dlog_g(h))
    """
    # factor n
    factors = factorint(n)

    # Modulare solving for each prime factor of n
    x_list = []
    modulo = []
    
    for q, e in factors.items():
        q_e = q ** e
        g_q = pow(g, n // q_e, p) 
        h_q = pow(h, n // q_e, p)  
        
        # Resolution over subfield of order q^e
        x_q = baby_giant_step(g_q, h_q, q_e, p)
        x_list.append(x_q)
        modulo.append(q_e)
    
    # Use the Chinese Remainder Theorem to construct the resultat
    x = chinese_remainder_theorem(x_list, modulo)
    
    return x

### Baby and Giand Step

In [17]:
def baby_giant_step(g: int, h: int, n: int, p: int) -> int:
    """
    Solve g^x ≡ h mod p.
    g: generator of the group
    h: the target value
    p: a prime number for modulo calculus
    n: the groupe order
    return: Logarithme discret x.
    """

    m = int(math.ceil(math.sqrt(n)))
    A = {}

    # Baby step
    for j in range(m):
        value = pow(g, j, p)
        A[value] = j

    # Giant step
    g_inv_m = pow(g, -m, p) 
    gamma = h
    for i in range(m):
        if gamma in A:
            return i * m + A[gamma]
        gamma = (gamma * g_inv_m) % p

    raise ValueError("Discret Logarithm not found.")

### Euclide etendu

In [29]:
def euclide(a: int, b: int) -> tuple:
    '''
        return (r0, u0, v0) such that a*u0 + b*v0 = gcd(a, b) = r0
    '''
    r0, r1 = a, b
    u0, u1 = 1, 0
    v0, v1 = 0, 1
    
    while r1 != 0:
        q = r0 // r1
        temp = r1
        r1 = r0 - q * r1
        r0 = temp

        utemp = u1
        u1 = u0 - q * u1
        u0 = utemp
        
        vtemp = v1
        v1 = v0 - q * v1
        v0 = vtemp
        
    return r0, u0, v0

### Chinese Remainder Theorem

In [30]:
def chinese_remainder_theorem(remainders: list, modulo: list) -> int:
    '''
        Solve a congruence system
        remainders: List of remeinders x_i.
        moduli: List of modulos m_i.
        Unique solution of x mod M, where M = product(m_i).
    '''
    x = 0
    M = 1
    for m in modulo:
        M *= m
    
    for r, m in zip(remainders, modulo):
        M_i = M // m
        inv = euclide(M_i, m)[1]
        x += r * M_i * inv
    
    return x % M

In [27]:
silver_pohlig_hellman(14,87203,75727617328661, 75727617328661-1) == 128405855

True

### Get secret key of receiver

In [38]:
def get_secret_key_of_receiver(group: tuple, modulo: int, public_key_of_receiver: int) -> int:
    '''
        group: (p, g) where p is a prime number and g a generator of the group (Z/pZ)*
        modulo: a number for modulo calculus
        public_key_of_receiver: the public key of receiver

        return the secret key of receiver
    '''
    p, g = group
    g_receiver = public_key_of_receiver
    secret_key = silver_pohlig_hellman(g=g, h=g_receiver, p=p, n=modulo)

    return secret_key

### El Gamal decryptage

In [32]:
def decryptage_El_Gamal(public_key: tuple, secret_key: int, encr_message: tuple) -> int:
    ''' public_key: the public key of receiver (p, g, g_receiver)
        encr_message: the encrypted message (u, v)
        
        return: decrypted message
    '''
    
    #Find the secret key of receiver: a
    p, g, x = public_key
    a = secret_key

    #Decryptage
    u, v = encr_message
    decr_message = (pow(u, -a, p) * v ) % p

    return decr_message

### Decode 

In [33]:
def decode(message: int) -> str:
    '''
        Decode the encoded message
        message: encoded message
        Return a decoded message
    '''
    
    conversion = {
        '11':'A', 
        '12':'B',
        '13':'C',
        '14':'D',
        '15':'E',
        '16':'F',
        '17':'G',
        '18':'H',
        '19':'I',
        '20':'J',
        '21':'K',
        '22':'L',
        '23':'M',
        '24':'N',
        '25':'O',
        '26':'P',
        '27':'Q',
        '28':'R',
        '29':'S',
        '30':'T',
        '31':'U',
        '32':'V',
        '33':'W',
        '34':'X',
        '35':'Y',
        '36':'Z',
        '41': ' ',
        '42': "'",
        '43': '.',
        '44': ',',
        '45': '?',
    }
    message = str(message)
    final_message = ''
    for i in range(0, len(message)//2):
        word = message[2*i]+message[2*i+1]
        final_message += conversion.get(word)
    return final_message 

## General decryptage

In [34]:
def decryptage_general(group: tuple, public_key: int, secret_key: int, list_message: list) -> str:
    '''
        group: (p, g) where p is a prime number and g a generator of the group (Z/pZ)*
        modulo: a number for modulo calculus
        public_key: the public key of receiver 
        secret_key: the secret key of receiver 
        list_message: list of crypted message 

        Return result: complete decoded message
    '''
    
    p, g = group
    modulo = p-1
    g_receiver = public_key
    result = ''
    for bloc in list_message:
        decr_message = decryptage_El_Gamal(public_key=(p, g, g_receiver), secret_key=secret_key, encr_message=bloc)
        result = result + decode(decr_message)
    return result

# Application

## Data we have

In [35]:
m1 = (83025882561049910713, 66740266984208729661)
m2 = (117087132399404660932, 44242256035307267278)
m3 = (67508282043396028407, 77559274822593376192) 
m4 = (60938739831689454113, 14528504156719159785)
m5 = (5059840044561914427, 59498668430421643612)
m6 = (92232942954165956522, 105988641027327945219)
m7 = (97102226574752360229, 46166643538418294423)
p = 123456789987654353003
g = 123456789
g_A = 52808579942366933355
g_B = 39318628345168608817

group = (p, g)
public_key = g_B
list_message = [m1, m2, m3, m4, m5, m6, m7]

## Secret key of receiver

In [39]:
secret_key = get_secret_key_of_receiver(group, modulo= p-1, public_key_of_receiver=public_key)
secret_key

5191

## The decrypted and decoded message

In [40]:
message = decryptage_general(group, public_key, secret_key, list_message)
message

'IN GALOIS FIELDS, FULL OF FLOWERS, PRIMITIVE ELEMENTS DANCE FOR HOURS.'